In [1]:
import turicreate


In [2]:
products=turicreate.SFrame('m_bfaa91c17752f745.frame_idx')

In [42]:
products
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [44]:
products['word_count']=turicreate.text_analytics.count_words(products['review'])

In [102]:
def awesome_count(word_count,word):
    if word in word_count:
        return word_count[word]
    else:
        return 0


In [103]:
for i in selected_words:
    
    products[i]=products['word_count'].apply(lambda word_count: awesome_count(word_count,i))

In [152]:
for i in selected_words:
    print(i+'\t\t',products[i].sum())
    

awesome		 3892.0
great		 55791.0
fantastic		 1664.0
amazing		 2628.0
love		 41994.0
horrible		 1110
bad		 4183
terrible		 1146.0
awful		 687
wow		 425
hate		 1107


In [105]:
giraffe_reviews = products[products['name']=='Vulli Sophie the Giraffe Teether']

In [106]:
products=products[products['rating']!=3]
products['sentiment']=products['rating']>=4
train_data,test_data=products.random_split(0.8,seed=0)

In [107]:
classification_model=turicreate.logistic_classifier.create(train_data,target='sentiment',features=['word_count'],validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 1.184181     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 1.371915     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 1.662834     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 2.658645     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [108]:
classification_model.evaluate(test_data,metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0    |        1.0         |        1.0         | 27976 | 5328 |
 |   1e-05   | 0.847972972972973  | 0.9975693451529882 | 27976 | 5328 |
 |   2e-05   | 0.829954954954955  | 0.9971761509865599 | 27976 | 5328 |
 |   3e-05   | 0.818506006006006  | 0.9969616814412353 | 27976 | 5328 |
 |   4e-05   | 0.8109984984984985 | 0.9967472118959108 | 27976 | 5328 |
 |   5e-05   | 0.8057432432432432 | 0.9966042321990277 | 27976 | 5328 |
 |   6e-05   | 0.7991741741741741 | 0.9962825278810409 | 27976 | 5328 |
 |   7e-05   | 0.7952327327327328 | 0.9961752931083786 | 27976 | 5328 |
 |   8e-05   | 0.7920420420420421 | 0.9961038032599371 | 27976 | 5328 |
 |   9e-05   | 0.7882882882882

In [109]:
for i in selected_words:
    i=i+'_count'
    print(i)
    


awesome_count
great_count
fantastic_count
amazing_count
love_count
horrible_count
bad_count
terrible_count
awful_count
wow_count
hate_count


In [110]:
model=turicreate.logistic_classifier.create(train_data,target='sentiment',features=selected_words,validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.157720     | 0.847708          | 0.846385            |

| 5         | 6        | 0.207335     | 0.847708          | 0.846385            |

| 6         | 7        | 0.240246     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [156]:
a=model.coefficients.sort('value',ascending=False)
a.print_rows(num_rows=12)

+-------------+-------+-------+-----------------------+----------------------+
|     name    | index | class |         value         |        stderr        |
+-------------+-------+-------+-----------------------+----------------------+
|     love    |  None |   1   |   1.359268866922512   | 0.02806830015209923  |
| (intercept) |  None |   1   |   1.3365913848877602  | 0.008929969787656582 |
|   awesome   |  None |   1   |   1.1335346660341417  | 0.08399643983187537  |
|   amazing   |  None |   1   |   1.1000933113660258  | 0.09954776260465989  |
|  fantastic  |  None |   1   |   0.885804756881427   | 0.11167591293399677  |
|    great    |  None |   1   |   0.8630655001196592  | 0.01895505244437695  |
|     wow     |  None |   1   | -0.009538236067681735 |  0.1604641122471166  |
|     bad     |  None |   1   |   -0.991477880065059  | 0.038484286646990606 |
|     hate    |  None |   1   |  -1.3484407222463144  | 0.07715698604297334  |
|    awful    |  None |   1   |  -2.0529082040313544

In [118]:
model.evaluate(test_data)

{'accuracy': 0.8463848186404036,
 'auc': 0.6936022046674926,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  159  |
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.3962265467087378,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 532

In [120]:
classification_model.evaluate(test_data)

{'accuracy': 0.9176975738650012,
 'auc': 0.9342357833151299,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      1       |        0        |  1344 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.33047871872408197,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.

In [126]:
products['prediction']=model.predict(products,output_type='probability')

In [127]:
products.sort('prediction',ascending=False)

name,review,rating,word_count,sentiment
"Baby K'tan Baby Carrier,Black, X-Large ...",Check and recheck theK'Tan for size issues ...,5.0,"{'seen': 1.0, '34': 2.0,'after': 1.0, 'katy': ...",1
Joovy Scooter SingleStroller Greenie ...,"***I've posted an UPDATEat the end***First, let ...",5.0,"{'compliments': 1.0,'many': 1.0, 'control': ...",1
"Britax ChaperoneStroller, Red Mill ...",I'm gonna give back bywriting a review. This ...,5.0,"{'later': 1.0, 'engaged':1.0, 'accidently': 2.0, ...",1
"Ju-Ju-Be Hobo Be DiaperBag, Dreamy Diamonds ...",Love love LOVE this bagand the Licorice Twirl ...,5.0,"{'13': 1.0, '20': 1.0,'11': 1.0, 'snatch': ...",1
Peg Perego Aria Stroller- Soleo ...,I literally have ownedalmost every stroller ...,4.0,"{'color': 1.0, 'such':1.0, 'travelling': 1.0, ...",1
Britax Advocate 70 CSClick &amp; Safe ...,"WE LOVE THIS CAR SEAT.Wehave 1 Cowmooflage, a ...",5.0,"{'laws': 1.0, 'kids':2.0, 'leading': 1.0, ...",1
"timi &amp; leslie Jessica7-Piece Diaper Bag Set, ...",Searched long and hardfor an un-diaper bag- ...,5.0,"{'so': 1.0, 'have': 1.0,'use': 1.0, 'good': 1.0, ...",1
"Britax B-Ready Stroller,Black ...",We just recentlypurchased this stroller ...,4.0,"{'perfectly': 1.0,'adaptor': 1.0, 'chos ...",1
"Graco Blossom Highchair,Townsend ...",We love this highchair.We have a 4 year old and ...,5.0,"{'older': 2.0, 'once':1.0, 'penny': 1.0, ...",1
"Peg-Perego 2010 Pliko P3Stroller, Java ...",I bought the Pilko P3 7years ago and used it ...,5.0,"{'prefer': 1.0,'awesome': 1.0, 'soho': ...",1


In [131]:
specific_product=products[products['name']=='Baby Trend Diaper Champ']
len(specific_product)

298

In [135]:
specific_product['sen_prediction']=classification_model.predict(specific_product,output_type='probability')

In [136]:
specific_product

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'convenient': 1.0,'more': 1.0, 'trash': ...",1
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'system': 1.0, 'try':1.0, 're': 1.0, 'still': ...",0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'nose': 1.0, 'for': 2.0,'investment': 1.0, ...",1
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'out': 1.0, 'pull': 1.0,'open': 1.0, 'pail': ...",1
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'winter': 1.0,'outside': 1.0, 'day': ...",1
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'mom': 1.0, 'my': 1.0,'empty': 2.0, 'poop': ...",1
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'yuck': 1.0, 'clean':1.0, 'all': 1.0, 'tra ...",0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1.0,'suggestions': 1.0, ...",1
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'best': 1.0, 'that':1.0, 'handle': 1.0, ' ...",1
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'no': 1.0, 'regular':1.0, 'part': 1.0, ...",1


In [141]:
y=specific_product.sort('sen_prediction',ascending=False)

In [142]:
y['sen_prediction'][0]

0.9999999999895941

In [148]:
x=model.predict(y,output_type='probability')
y['mod_pred']=model.predict(y,output_type='probability')

In [149]:
x.sort(ascending=False)

dtype: float
Rows: 298
[0.9981253623335121, 0.9955677154227357, 0.9955677154227357, 0.9883959649944543, 0.9881322423088145, 0.9881322423088145, 0.9881322423088145, 0.9829620803909053, 0.9829620803909053, 0.9829620803909053, 0.9829620803909053, 0.9829620803909053, 0.9829620803909053, 0.9829620803909053, 0.9829620803909053, 0.972318678885589, 0.972318678885589, 0.972318678885589, 0.972318678885589, 0.972318678885589, 0.972318678885589, 0.972318678885589, 0.972318678885589, 0.972318678885589, 0.972318678885589, 0.972318678885589, 0.972318678885589, 0.972318678885589, 0.9562890526658798, 0.9553286211620002, 0.9553286211620002, 0.9553286211620002, 0.9553286211620002, 0.9553286211620002, 0.9553286211620002, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.936781924479937, 0.93

In [154]:
y

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,I read a review belowthat can explain exactly ...,4.0,"{'key': 1.0, 'have': 1.0,'pieces': 1.0, 'betwe ...",1
Baby Trend Diaper Champ,I have never written areview for Amazon but I ...,5.0,"{'priceless': 1.0,'knows': 1.0, 'parent': ...",1
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1.0,'suggestions': 1.0, ...",1
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'around': 1.0, 'any':1.0, 't': 1.0, 'isn': ...",1
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'either': 1.0, 'be':1.0, 't': 1.0, 'not': ...",1
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'hope': 1.0, 'make':1.0, 'slower': 1.0, ...",1
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'buy': 1.0, 'product':1.0, 'recommend': 1.0, ...",1
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5.0,"{'ll': 1.0, 'baby': 1.0,'recommended': 1.0, ' ...",1
Baby Trend Diaper Champ,I see that there arecomplaints of stinkiness ...,5.0,"{'very': 1.0, 'told':1.0, 'all': 1.0, ...",1
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'sorry': 1.0, 'be': 1.0,'you': 2.0, 'sell': 1.0, ...",1
